<a href="https://colab.research.google.com/github/NeerajMehta15/RAG_Implementation/blob/main/RAG_reading_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain
!pip install -U langchain-community
!pip install pypdf
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [32]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
import torch
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [26]:
Loader = PyPDFLoader('/content/drive/MyDrive/Datasets/Test_rag.pdf')
data = Loader.load()
print(data[0])

page_content='International Journal of Management Studies          ISSN(Print) 2249-0302 ISSN (Online)2231-2528 
http://www.researchersworld.com/ijms/ 
 
V ol.–V , Special Issue - 5, August 2018 [9] 
Gangotri), Mandakini (in Kedarnath) and Alaknanda (in Badrinath).Traditionally, the Char Dharm yatra (or 
excursion) is done from the west to the east - beginning from Yamunotri, then continuing to Gangotri lastly to 
Kedarnath and Badrinath. In spite of the fact that the Char (four) Dhams are open through a system of 
motorable streets, burdensome trails remain in the antiquated havens of confidence for a satisfying experience. 
The other favored goals are Haridwar, Nanakmatta, Meetha -Reetha Sahib, Piran Kaliyar and Punyagiri. Among 
the different hypnotizing religious customs, the night Aarti by the banks of the Ganga at Haridwar is in itself an 
awesome ordeal. The Yatras to explor er goals, for example, Nanda Devi and Kailash Manasarover offer a 
chance to relish some of the most stun

In [27]:
# Join all the text content into one large string
full_text = " ".join([doc.page_content for doc in data])

# Define chunking parameters
chunk_size = 500
chunk_overlap = 50

# Initialize the text splitter
ct_splitter = CharacterTextSplitter(separator='.', chunk_size=chunk_size, chunk_overlap=chunk_overlap)

# Split the text into chunks
docs = ct_splitter.split_text(full_text)

# Print the chunks and their lengths
print(docs)
print([len(doc) for doc in docs])


['International Journal of Management Studies          ISSN(Print) 2249-0302 ISSN (Online)2231-2528 \nhttp://www.researchersworld.com/ijms/ \n \nV ol.–V , Special Issue - 5, August 2018 [9] \nGangotri), Mandakini (in Kedarnath) and Alaknanda (in Badrinath).Traditionally, the Char Dharm yatra (or \nexcursion) is done from the west to the east - beginning from Yamunotri, then continuing to Gangotri lastly to \nKedarnath and Badrinath', 'In spite of the fact that the Char (four) Dhams are open through a system of \nmotorable streets, burdensome trails remain in the antiquated havens of confidence for a satisfying experience. \nThe other favored goals are Haridwar, Nanakmatta, Meetha -Reetha Sahib, Piran Kaliyar and Punyagiri. Among \nthe different hypnotizing religious customs, the night Aarti by the banks of the Ganga at Haridwar is in itself an \nawesome ordeal', 'The Yatras to explor er goals, for example, Nanda Devi and Kailash Manasarover offer a \nchance to relish some of the most s

In [29]:
print(type(docs))  # Print the type of docs

# Print each chunk with a label
for i, chunk in enumerate(docs[:5], start=1):  # Adjust the range as needed
    print(f"Chunk {i}:")
    print(chunk)
    print("\n" + "-"*50 + "\n")  # Separator for readability


<class 'list'>
Chunk 1:
International Journal of Management Studies          ISSN(Print) 2249-0302 ISSN (Online)2231-2528 
http://www.researchersworld.com/ijms/ 
 
V ol.–V , Special Issue - 5, August 2018 [9] 
Gangotri), Mandakini (in Kedarnath) and Alaknanda (in Badrinath).Traditionally, the Char Dharm yatra (or 
excursion) is done from the west to the east - beginning from Yamunotri, then continuing to Gangotri lastly to 
Kedarnath and Badrinath

--------------------------------------------------

Chunk 2:
In spite of the fact that the Char (four) Dhams are open through a system of 
motorable streets, burdensome trails remain in the antiquated havens of confidence for a satisfying experience. 
The other favored goals are Haridwar, Nanakmatta, Meetha -Reetha Sahib, Piran Kaliyar and Punyagiri. Among 
the different hypnotizing religious customs, the night Aarti by the banks of the Ganga at Haridwar is in itself an 
awesome ordeal

--------------------------------------------------

Chu

In [31]:
# Building a vector database
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize the HuggingFace embeddings model
embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Wrap each chunk of text in a Document object
documents = [Document(page_content=doc) for doc in docs]

# Create the vector store
vectorstore = Chroma.from_documents(documents,  # Use `documents` here
                                    embedding=embedding_function,
                                    persist_directory='/content/drive/MyDrive/Datasets/')

# Convert the database to a retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})


In [33]:
# Define the message
message = """
Give me the list of holy places mentioned in the documents
"""

# Create the prompt template
prompt_template = ChatPromptTemplate.from_messages([("human", message)])


In [36]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Load a smaller free LLM model from Hugging Face
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create a pipeline for text generation
hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=100)

# Wrap the pipeline in LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=hf_pipeline)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu
<ipython-input-36-c3e26e32f7f7>:13: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [47]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Load a smaller free LLM model from Hugging Face
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create a pipeline for text generation
hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=100)

# Wrap the pipeline in LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=hf_pipeline)


Device set to use cpu


In [48]:
#Chaining it all togther
from langchain_core.runnables import RunnablePassthrough
rag_chain = ({"guidelines": retriever, "copy": RunnablePassthrough()}
| prompt_template
| llm)
response = rag_chain.invoke("What are the four char dham mentioned")



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [49]:
print(response)

Human: 
Give me the list of holy places mentioned in the documents
Give me a list of holy places mentioned in the documents
Grant me a list of holy places mentioned in the files
Keep in mind the exact number of times an icon will appear in your project
Give me a list of holy places mentioned in the documents
If I don't include the names of specific icons, I'll be missing a particular icon that will only appear there
My project is the same icon: http
